# OMIE Daily Table Ingestion




Creates Delta tables from OMIE Bronze layer Parquet files.




**Input:** Parquet files in `Files/bronze/OMIE/{year}/`  


**Output:** 3 Delta tables (one per year): `brz_omie_daily_2023`, `brz_omie_daily_2024`, `brz_omie_daily_2025`




Following the same pattern as OPEN-METEO colleagues.

In [3]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import json, os

spark = SparkSession.builder.getOrCreate()

print("🚀 OMIE Daily Table Ingestion Started")
print(f"✅ Spark session active: {spark.version}")

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# -------------------------------
# Recibe parámetros del pipeline
# -------------------------------
# Esperamos que la primera actividad devuelva JSON con:
# {"start_date": "...", "end_date": "...", "years_touched": [2023,2024,2025]}

try:
    dbutils.widgets.text("updated_json", "")
    updated_json = dbutils.widgets.get("updated_json")
    
    if not updated_json.strip():
        print("⚠️  No se recibieron parámetros del pipeline. Usando configuración por defecto.")
        # Configuración por defecto: procesar todos los años
        years_to_process = [2023, 2024, 2025]
    else:
        payload = json.loads(updated_json)
        years_to_process = payload.get("years_touched", [2023, 2024, 2025])
        
except:
    # Fallback para ejecución local/manual
    print("💻 Ejecución local detectada. Procesando todos los años.")
    years_to_process = [2023, 2024, 2025]
    
if not years_to_process:
    print("❌ No hay años para procesar. Terminando notebook.")
    try:
        dbutils.notebook.exit("No hay trabajo")
    except:
        print("Terminando ejecución local.")
        
print(f"🔹 Años a procesar: {years_to_process}")

In [ ]:
# -------------------------------
# Construir rutas base según años
# -------------------------------

# Detectar el entorno y ajustar las rutas
try:
    # Intentar detectar si estamos en un lakehouse específico
    current_lakehouse = mssparkutils.fs.ls("/")
    
    # En Fabric, usar rutas relativas al lakehouse actual
    BASE_DIR = "/Files/bronze/OMIE"
    base_paths = [f"{BASE_DIR}/{str(y)}" for y in years_to_process]
    
    print(f"📁 Directorio base (Fabric): {BASE_DIR}")
    print(f"🏢 Entorno: Microsoft Fabric Lakehouse")
    
except:
    # Fallback para desarrollo local
    BASE_DIR = "Files/bronze/OMIE"
    base_paths = [os.path.join(BASE_DIR, str(y)) for y in years_to_process]
    
    print(f"📁 Directorio base (Local): {BASE_DIR}")
    print(f"💻 Entorno: Desarrollo local")

print(f"📂 Rutas a procesar: {base_paths}")

# Función mejorada para verificar rutas
def check_path_safely(path):
    """Verifica una ruta de forma segura con múltiples métodos"""
    try:
        # Método 1: Usar mssparkutils.fs.ls
        files = mssparkutils.fs.ls(path)
        parquet_files = [f for f in files if f.name.endswith(".parquet")]
        return True, len(parquet_files), "mssparkutils"
    
    except Exception as e1:
        try:
            # Método 2: Usar spark directamente para verificar si hay archivos
            test_pattern = f"{path}/*.parquet"
            df_test = spark.read.parquet(test_pattern)
            # Si llegamos aquí, hay al menos un archivo Parquet
            file_count = len(mssparkutils.fs.ls(path)) if path.startswith("/") else 1
            return True, file_count, "spark_direct"
        
        except Exception as e2:
            # Método 3: Verificar con diferentes formatos de path
            alternative_paths = []
            
            if not path.startswith("/"):
                alternative_paths.append(f"/{path}")
            if not path.startswith("abfss://"):
                alternative_paths.append(f"Files/bronze/OMIE/{os.path.basename(path)}")
            
            for alt_path in alternative_paths:
                try:
                    files = mssparkutils.fs.ls(alt_path)
                    parquet_files = [f for f in files if f.name.endswith(".parquet")]
                    if parquet_files:
                        return True, len(parquet_files), f"alternative:{alt_path}"
                except:
                    continue
            
            return False, 0, f"failed: {str(e1)[:100]}"

# Verificar que existen las rutas
available_paths = []
for path in base_paths:
    year = os.path.basename(path)
    print(f"\n🔍 Verificando año {year}: {path}")
    
    is_available, file_count, method = check_path_safely(path)
    
    if is_available and file_count > 0:
        available_paths.append((path, file_count))
        print(f"✅ {path}: {file_count} archivos Parquet (método: {method})")
    elif is_available and file_count == 0:
        print(f"⚠️  {path}: Directorio accesible pero sin archivos Parquet (método: {method})")
    else:
        print(f"❌ {path}: No accesible - {method}")

if not available_paths:
    print("\n❌ No se encontraron archivos Parquet en ninguna ruta.")
    print("💡 Posibles soluciones:")
    print("   1. Verificar que el notebook omie.ipynb se ha ejecutado correctamente")
    print("   2. Confirmar que estás en el lakehouse correcto")
    print("   3. Verificar las rutas de los archivos Parquet")
    
    # Intentar mostrar lo que hay en el directorio bronze base
    try:
        bronze_base = "/Files/bronze/OMIE" if BASE_DIR.startswith("/") else "Files/bronze/OMIE"
        print(f"\n🔍 Contenido del directorio base {bronze_base}:")
        base_files = mssparkutils.fs.ls(bronze_base)
        for f in base_files:
            print(f"   {'📁' if f.isDir else '📄'} {f.name}")
    except Exception as e:
        print(f"   ❌ No se pudo acceder al directorio base: {e}")
    
    try:
        dbutils.notebook.exit("Sin archivos")
    except:
        print("Terminando ejecución local.")

print(f"\n🎯 Rutas válidas encontradas: {len(available_paths)}")

In [ ]:
# -------------------------------
# Diagnóstico del entorno Fabric y verificación de prerequisitos
# -------------------------------

print("🔍 Diagnosticando entorno de Microsoft Fabric...")

# Variables para tracking del diagnóstico
bronze_data_exists = False
years_with_data = []
lakehouse_root = None

try:
    # Verificar directorio raíz
    root_contents = mssparkutils.fs.ls("/")
    print(f"📂 Contenido directorio raíz:")
    for item in root_contents[:10]:  # Mostrar solo los primeros 10
        print(f"   {'📁' if item.isDir else '📄'} {item.name}")
    
    # En Fabric, buscar el lakehouse correcto explorando los directorios GUID
    print(f"\n🔍 Buscando estructura lakehouse en directorios GUID...")
    
    # Verificar si existe Files directamente en root
    files_exists = any(item.name == "Files" and item.isDir for item in root_contents)
    
    if files_exists:
        lakehouse_root = "/Files"
        print(f"✅ Encontrado Files en root: {lakehouse_root}")
    else:
        # Buscar Files dentro de los directorios GUID (lakehouses)
        for guid_item in root_contents:
            if guid_item.isDir:
                try:
                    guid_contents = mssparkutils.fs.ls(f"/{guid_item.name}")
                    if any(item.name == "Files" and item.isDir for item in guid_contents):
                        lakehouse_root = f"/{guid_item.name}/Files"
                        print(f"✅ Encontrado lakehouse en: {lakehouse_root}")
                        break
                except:
                    continue
    
    if lakehouse_root:
        print(f"📁 Usando lakehouse root: {lakehouse_root}")
        
        # Verificar contenido de Files
        files_contents = mssparkutils.fs.ls(lakehouse_root)
        print(f"📂 Contenido de {lakehouse_root}:")
        for item in files_contents:
            print(f"   {'📁' if item.isDir else '📄'} {item.name}")
        
        # Verificar si existe bronze
        bronze_exists = any(item.name == "bronze" and item.isDir for item in files_contents)
        print(f"📁 Directorio 'bronze' existe: {'✅' if bronze_exists else '❌'}")
        
        if bronze_exists:
            # Verificar contenido de bronze
            bronze_path = f"{lakehouse_root}/bronze"
            bronze_contents = mssparkutils.fs.ls(bronze_path)
            print(f"📂 Contenido de {bronze_path}:")
            for item in bronze_contents:
                print(f"   {'📁' if item.isDir else '📄'} {item.name}")
            
            # Verificar si existe OMIE
            omie_exists = any(item.name == "OMIE" and item.isDir for item in bronze_contents)
            print(f"📁 Directorio 'OMIE' existe: {'✅' if omie_exists else '❌'}")
            
            if omie_exists:
                # Verificar contenido de OMIE
                omie_path = f"{bronze_path}/OMIE"
                omie_contents = mssparkutils.fs.ls(omie_path)
                print(f"📂 Contenido de {omie_path}:")
                
                for item in omie_contents:
                    print(f"   {'📁' if item.isDir else '📄'} {item.name}")
                    
                    # Si es directorio numérico (año), verificar su contenido
                    if item.isDir and item.name.isdigit():
                        try:
                            year_path = f"{omie_path}/{item.name}"
                            year_contents = mssparkutils.fs.ls(year_path)
                            parquet_count = sum(1 for f in year_contents if f.name.endswith(".parquet"))
                            total_files = len(year_contents)
                            
                            print(f"      📦 {item.name}: {total_files} archivos total, {parquet_count} Parquet")
                            
                            if parquet_count > 0:
                                years_with_data.append(int(item.name))
                                bronze_data_exists = True
                                
                                # Mostrar algunos archivos de ejemplo
                                parquet_files = [f.name for f in year_contents if f.name.endswith(".parquet")][:3]
                                print(f"         Ejemplos: {', '.join(parquet_files)}")
                            
                        except Exception as e:
                            print(f"      ❌ {item.name}: Error accediendo - {str(e)[:50]}...")
            else:
                print("❌ No se encontró el directorio OMIE en bronze")
        else:
            print("❌ No se encontró el directorio bronze")
    else:
        print("❌ No se encontró estructura lakehouse válida")
        
        # Como fallback, intentar buscar directamente por los paths que mencionaste
        print(f"\n🔍 Intentando paths alternativos mencionados por el usuario...")
        alternative_paths = [
            "Files/bronze/OMIE",
            "/Files/bronze/OMIE"
        ]
        
        for alt_path in alternative_paths:
            try:
                print(f"   🔍 Probando: {alt_path}")
                omie_contents = mssparkutils.fs.ls(alt_path)
                
                print(f"   ✅ Acceso exitoso a {alt_path}")
                lakehouse_root = "/Files" if alt_path.startswith("/") else "Files"
                
                for item in omie_contents:
                    print(f"      {'📁' if item.isDir else '📄'} {item.name}")
                    
                    if item.isDir and item.name.isdigit():
                        try:
                            year_path = f"{alt_path}/{item.name}"
                            year_contents = mssparkutils.fs.ls(year_path)
                            parquet_count = sum(1 for f in year_contents if f.name.endswith(".parquet"))
                            
                            if parquet_count > 0:
                                years_with_data.append(int(item.name))
                                bronze_data_exists = True
                                print(f"         ✅ {item.name}: {parquet_count} archivos Parquet")
                                
                        except Exception as e:
                            print(f"         ❌ {item.name}: {str(e)[:50]}...")
                
                break  # Si encontramos uno que funciona, parar
                
            except Exception as e:
                print(f"   ❌ {alt_path}: {str(e)[:50]}...")
                continue

except Exception as e:
    print(f"❌ Error en diagnóstico: {e}")
    print("💡 Esto puede indicar que el notebook no está ejecutándose en un lakehouse válido")

# Resultado del diagnóstico
print(f"\n📊 Resultado del diagnóstico:")
print(f"   🎯 Datos Bronze OMIE encontrados: {'✅' if bronze_data_exists else '❌'}")
print(f"   📅 Años con datos: {sorted(years_with_data) if years_with_data else 'Ninguno'}")
print(f"   📁 Lakehouse root: {lakehouse_root if lakehouse_root else 'No detectado'}")

if not bronze_data_exists:
    print(f"\n🚨 PROBLEMA DETECTADO:")
    print(f"   ❌ No se encontraron archivos Parquet en el directorio Bronze OMIE")
    print(f"   💡 SOLUCIÓN:")
    print(f"      1. Verificar que estás en el lakehouse correcto")
    print(f"      2. Si los datos existen, ajustar las rutas en el siguiente paso")
    print(f"      3. O ejecutar el notebook 'omie.ipynb' para crear los datos Bronze")
    
    # No terminar automáticamente, permitir ajustes manuales
    print(f"\n⚠️  Continuando para permitir ajustes manuales de rutas...")
        
else:
    print(f"\n✅ Prerequisitos cumplidos!")
    print(f"   📦 Datos Bronze disponibles para años: {sorted(years_with_data)}")
    print(f"   🎯 Listo para crear tablas Delta")

# Hacer disponible el lakehouse root para celdas posteriores
if lakehouse_root:
    globals()['DETECTED_LAKEHOUSE_ROOT'] = lakehouse_root

print(f"\n🏁 Diagnóstico completado")

In [ ]:
# -------------------------------




# Escritura de tablas Delta por año (robusto, con ABFSS GUID)




# -------------------------------








from pyspark.sql import functions as F


import re








# Utilidades para resolver ruta ABFSS con GUIDs y evitar FriendlyNameSupportDisabled




try:




    from notebookutils import mssparkutils  # Disponible en Fabric




except Exception:




    mssparkutils = None








# Overrides manuales opcionales (rellenar si la auto-detección falla)




WORKSPACE_ID_OVERRIDE = None  # p.ej. "ecf938c4-c449-48de-a07c-1d968a72b3d1"




LAKEHOUSE_ID_OVERRIDE = None  # p.ej. "12345678-aaaa-bbbb-cccc-1234567890ab"








def _get_env_id(getter_names):




    if not mssparkutils:




        return None




    for name in getter_names:




        try:




            fn = getattr(mssparkutils.env, name, None)




            if callable(fn):




                val = fn()




                if val:




                    return val




        except Exception:




            pass




    return None








def resolve_onelake_abfss(rel_path: str) -> str:




    """Devuelve ruta ABFSS con GUIDs si es posible, si no, devuelve la original."""




    ws_id = WORKSPACE_ID_OVERRIDE or _get_env_id(["getWorkspaceId", "getWorkspaceGUID", "getWorkspaceGuid"]) 




    lakehouse_id = LAKEHOUSE_ID_OVERRIDE or _get_env_id(["getLakehouseId", "getArtifactId", "getItemId"]) 




    if ws_id and lakehouse_id and rel_path:




        rel_path = rel_path.replace("\\", "/").lstrip("/")




        return f"abfss://{ws_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}/{rel_path}"




    return rel_path








# Helpers: sanitizar nombres de columnas para Delta




def _sanitize_col_name(name: str) -> str:




    # a minúsculas




    n = (name or "").strip().lower()




    # reemplazar caracteres inválidos para Delta: espacios, comas, punto y coma, llaves, paréntesis, tabs, saltos, igual y también dos puntos




    n = re.sub(r"[\s,;{}()=\n\t:]+", "_", n)




    # quitar guiones bajos al inicio/fin




    n = n.strip("_")




    # si empieza por número, prefijar




    if n and n[0].isdigit():




        n = f"c_{n}"




    # fallback por si queda vacío




    return n or "column"








def sanitize_columns(cols):




    seen = {}




    new_cols = []




    for c in cols:




        base = _sanitize_col_name(c)




        candidate = base




        # asegurar unicidad




        while candidate in seen:




            seen[base] = seen.get(base, 1) + 1




            candidate = f"{base}_{seen[base]}"




        seen[candidate] = 1




        new_cols.append(candidate)




    return new_cols








# Años a procesar




years = years_to_process if 'years_to_process' in locals() and years_to_process else [2023, 2024, 2025]








# Directorio base friendly y ABFSS




friendly_base = "Files/bronze/OMIE"




BASE_DIR_ABFSS = resolve_onelake_abfss(friendly_base)




print(f"📁 Base (ABFSS): {BASE_DIR_ABFSS}")








# Lista de resultados para el resumen final




if 'tables_created' not in locals():




    tables_created = []








total_records_processed = 0








for year in sorted(set(int(y) for y in years)):




    pattern = f"{BASE_DIR_ABFSS}/{year}/*.parquet"




    print(f"\n📦 Año {year} | Leyendo patrón: {pattern}")








    # Lectura primaria con patrón (ABFSS GUID)




    try:




        df_raw = spark.read.parquet(pattern)




    except Exception as e:




        print(f"   ⚠️  Falló lectura por patrón: {e}\n   ▶️  Intentando fallback por lista de archivos...")




        # Fallback: intentar listar archivos y leer de una lista explícita




        try:




            year_dir = f"{BASE_DIR_ABFSS}/{year}"




            files = [f.path for f in mssparkutils.fs.ls(year_dir) if f.name.endswith('.parquet')] if mssparkutils else []




            if not files:




                # Segundo fallback: intentar la ruta friendly solo para listar




                friendly_dir = f"/{friendly_base}/{year}"




                files = [f.path for f in mssparkutils.fs.ls(friendly_dir) if f.name.endswith('.parquet')] if mssparkutils else []




            if files:




                print(f"   ℹ️  Cargando {len(files)} archivo(s) explícitos")




                df_raw = spark.read.parquet(files)




            else:




                print("   ❌ No se encontraron archivos para fallback")




                continue




        except Exception as e2:




            print(f"   ❌ Fallback por lista de archivos también falló: {e2}")




            continue








    # Normalizar y sanitizar columnas antes de escribir a Delta




    original_cols = df_raw.columns




    sanitized_cols = sanitize_columns(original_cols)




    if sanitized_cols != original_cols:




        print("   🔤 Renombrando columnas para compatibilidad Delta:")




        for oc, nc in zip(original_cols, sanitized_cols):




            if oc != nc:




                print(f"      • '{oc}' -> '{nc}'")




    df = df_raw.toDF(*sanitized_cols)




    # Validación final de nombres: si queda alguno inválido, re-renombrar forzosamente


    invalid_pattern = r"[\s,;{}()=\n\t:]+"


    remaining_invalid = [c for c in df.columns if re.search(invalid_pattern, c)]


    if remaining_invalid:


        print(f"   ⚠️ Persisten nombres inválidos: {remaining_invalid} -> aplicando renombrado forzoso")


        current_cols = list(df.columns)


        rename_map = {}


        used = set()



        def _unique(target):


            base = target


            i = 1


            while base in used:


                base = f"{target}_{i}"


                i += 1


            used.add(base)


            return base




        for oc in current_cols:


            nc = _sanitize_col_name(oc)


            nc = _unique(nc)


            rename_map[oc] = nc


        for oc, nc in rename_map.items():


            if oc != nc:


                df = df.withColumnRenamed(oc, nc)


        print(f"   ✅ Columnas finales: {df.columns}")


    else:


        print(f"   ✅ Columnas finales: {df.columns}")








    # Precio OMIE




    if 'marginalpdbc' in df.columns:




        df = df.withColumn('marginal_price_eur_mwh', F.col('marginalpdbc').cast('double'))




    elif 'marginal_price_eur_mwh' not in df.columns:




        df = df.withColumn('marginal_price_eur_mwh', F.lit(None).cast('double'))








    # Año y fecha de extracción




    df = df.withColumn('extraction_year', F.lit(int(year)))




    if 'extraction_date' in df.columns:




        df = df.withColumn('extraction_date_parsed', F.col('extraction_date').cast('string'))




    elif 'extraction_date_parsed' not in df.columns:




        df = df.withColumn('extraction_date_parsed', F.regexp_extract(F.input_file_name(), r'(20\d{6})', 1))








    # Calidad y metadatos




    df = (




        df




        .withColumn('price_valid', (F.col('marginal_price_eur_mwh').isNotNull()) & (F.col('marginal_price_eur_mwh') >= 0))




        .withColumn('price_category', F.when(F.col('marginal_price_eur_mwh') < 50, F.lit('LOW'))




                                      .when(F.col('marginal_price_eur_mwh') < 150, F.lit('MID'))




                                      .otherwise(F.lit('HIGH')))




        .withColumn('data_quality_score', F.when(F.col('price_valid'), F.lit(1.0)).otherwise(F.lit(0.5)))




        .withColumn('table_created_at', F.current_timestamp())




        .withColumn('source_file', F.input_file_name())




        .withColumn('year', F.col('extraction_year'))




    )








    table_name = f"brz_omie_daily_{year}"




    print(f"   💾 Guardando como tabla Delta: {table_name}")








    (




        df.write.format('delta')




          .mode('overwrite')




          .option('overwriteSchema', 'true')




          .partitionBy('year')




          .saveAsTable(table_name)




    )








    # Contar y acumular




    try:




        recs = spark.table(table_name).count()




    except Exception:




        recs = 0








    total_records_processed += recs




    tables_created.append({




        'table_name': table_name,




        'year': int(year),




        'records': recs,




        'source_files': pattern,




        'total_files_available': 'N/A',




        'processing_status': 'Tabla creada'




    })








print(f"\n✅ Tablas creadas/actualizadas: {len(tables_created)} | Registros totales: {total_records_processed:,}")

In [ ]:
# -------------------------------
# Resumen final y validación
# -------------------------------

from pyspark.sql import functions as F

# Si no hay tables_created del paso anterior, buscar tablas existentes
if 'tables_created' not in locals() or not tables_created:
    print("🔍 Buscando tablas OMIE existentes...")
    
    # Buscar tablas que ya existen
    try:
        existing_tables = spark.sql("SHOW TABLES").collect()
        omie_tables = [row.tableName for row in existing_tables if row.tableName.startswith('brz_omie_daily_')]
        
        if omie_tables:
            print(f"✅ Encontradas {len(omie_tables)} tablas OMIE existentes:")
            for table in sorted(omie_tables):
                print(f"   📅 {table}")
            
            # Reconstruir la lista tables_created para el resumen
            tables_created = []
            total_records_processed = 0
            
            for table_name in omie_tables:
                try:
                    year = int(table_name.split('_')[-1])  # Extraer año del nombre
                    count = spark.table(table_name).count()
                    total_records_processed += count
                    
                    tables_created.append({
                        "table_name": table_name,
                        "year": year,
                        "records": count,
                        "source_files": "N/A (tabla existente)",
                        "total_files_available": "N/A (tabla existente)",
                        "processing_status": "Tabla ya existía"
                    })
                except Exception as e:
                    print(f"   ⚠️  Error accediendo a {table_name}: {e}")
        else:
            print("❌ No se encontraron tablas OMIE existentes")
            tables_created = []
            total_records_processed = 0
            
    except Exception as e:
        print(f"❌ Error buscando tablas existentes: {e}")
        tables_created = []
        total_records_processed = 0

if tables_created:
    print(f"\n📋 Resumen de tablas OMIE creadas:")
    
    for table_info in sorted(tables_created, key=lambda t: t['year']):
        table_name = table_info["table_name"]
        year = table_info["year"]
        records = table_info["records"]
        source_files = table_info["source_files"]
        
        print(f"\n📅 {table_name}:")
        print(f"   📊 Registros: {records:,}")
        print(f"   📦 Archivos origen: {source_files}")
        
        # Validación rápida de la tabla
        try:
            table_df = spark.table(table_name)
            
            # Estadísticas básicas
            quality_stats = table_df.select(
                F.avg("data_quality_score").alias("avg_quality"),
                F.count("source_file").alias("total_records"),
                F.countDistinct("source_file").alias("unique_files"),
                F.min("extraction_date_parsed").alias("min_date"),
                F.max("extraction_date_parsed").alias("max_date")
            ).collect()[0]
            
            print(f"   🎯 Calidad promedio: {quality_stats['avg_quality']:.2f}")
            print(f"   📁 Archivos únicos: {quality_stats['unique_files']}")
            
            if quality_stats['min_date'] and quality_stats['max_date']:
                print(f"   📅 Rango fechas: {quality_stats['min_date']} a {quality_stats['max_date']}")
            
            # Mostrar estadísticas de precios para OMIE
            if 'marginal_price_eur_mwh' in [field.name for field in table_df.schema.fields]:
                price_stats = table_df.select(
                    F.min("marginal_price_eur_mwh").alias("min_price"),
                    F.max("marginal_price_eur_mwh").alias("max_price"),
                    F.avg("marginal_price_eur_mwh").alias("avg_price"),
                    F.countDistinct("price_category").alias("price_categories")
                ).collect()[0]
                
                print(f"   💰 Precio min: {price_stats['min_price']:.2f} €/MWh")
                print(f"   💰 Precio max: {price_stats['max_price']:.2f} €/MWh")
                print(f"   💰 Precio promedio: {price_stats['avg_price']:.2f} €/MWh")
                print(f"   📊 Categorías de precio: {price_stats['price_categories']}")
            
        except Exception as e:
            print(f"   ⚠️  Error en validación: {e}")
    
    # Crear tabla resumen
    print(f"\n📊 Estadísticas consolidadas:")
    print(f"   📁 Total tablas: {len(tables_created)}")
    print(f"   📅 Años cubiertos: {sorted([t['year'] for t in tables_created])}")
    print(f"   📈 Total registros: {total_records_processed:,}")
    
    # Verificar que todas las tablas son accesibles
    print(f"\n🔍 Verificación de acceso a tablas:")
    for table_info in tables_created:
        try:
            spark.sql(f"SELECT COUNT(*) FROM {table_info['table_name']}").collect()
            print(f"   ✅ {table_info['table_name']}: Accesible")
        except Exception as e:
            print(f"   ❌ {table_info['table_name']}: Error - {e}")
    
    # Mostrar muestra de datos de la primera tabla
    if tables_created:
        sample_table = sorted(tables_created, key=lambda t: t['year'])[0]["table_name"]
        print(f"\n🔍 Muestra de datos de {sample_table}:")
        try:
            spark.table(sample_table).select(
                "source_file", "extraction_year", "extraction_date_parsed", 
                "marginal_price_eur_mwh", "price_category", "price_valid",
                "data_quality_score", "table_created_at"
            ).show(5, truncate=False)
        except Exception as e:
            print(f"   ⚠️  Error mostrando muestra: {e}")
            # Fallback con columnas básicas
            try:
                spark.table(sample_table).show(3, truncate=False)
            except Exception as e2:
                print(f"   ❌ Error con fallback: {e2}")
    
    result_message = f"Años procesados: {[t['year'] for t in tables_created]}"
    
else:
    print("❌ No se crearon tablas")
    result_message = "Sin tablas creadas"

print(f"\n🏁 Finalizando notebook...")
print(f"📝 Resultado: {result_message}")

# Salida del notebook para pipeline
try:
    dbutils.notebook.exit(result_message)
except:
    print(f"✅ Ejecución local completada: {result_message}")